Introduction

Importing libraries pandas, numpy, matplotlib, seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Reading sales table data

In [ ]:
sales_table = pd.read_csv('../data/generated/sales_data_sample_CLEAN.csv', encoding='CP1252')

Exploring orders stats

In [ ]:
round(sales_table['ORDERLINENUMBER'].mean(), 2)

Average line number per order is 6.47

In [ ]:
round((sales_table.groupby(['ORDERNUMBER'])['SALES'].sum()).mean(), 2)

Average sales per order is 32679.57

Checking year to year sales

In [ ]:
sales_per_year = sales_table.groupby(['YEAR_ID'])['SALES'].sum()
sales_per_year

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=sales_per_year)
plt.title('')
plt.xticks(rotation=45)

for i, (year, sales) in enumerate(sales_per_year.items()):
    plt.text(i, sales, f"${sales:,.0f}", 
             ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
sales_table.loc[sales_table['YEAR_ID']==2005, 'MONTH_ID'].unique()

Year 2005 data is only till May

Projecting 2005 for the end of the year sales

In [ ]:
sales_per_year[2005] = sales_per_year[2005] * (12/5)
sales_per_year

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=sales_per_year)
plt.title('')
plt.xticks(rotation=45)

for i, (year, sales) in enumerate(sales_per_year.items()):
    plt.text(i, sales, f"${sales:,.0f}", 
    ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
sales_per_quarter = sales_table.groupby(['YEAR_ID', 'QTR_ID'])['SALES'].sum().reset_index()
sales_per_quarter

Looks like there was rising in sales 2003 -> 2004, but then 2005 sales is projected to be a bit lower than 2004 sales

In [ ]:
sales_per_quarter['Quarter'] = 'Q' + sales_per_quarter['QTR_ID'].astype(str)

plt.figure(figsize=(12, 6))
ax = sns.barplot(data=sales_per_quarter, x='Quarter', y='SALES', hue='YEAR_ID', palette='Set2')

plt.title('Quarterly Sales: Year-over-Year Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Quarter', fontsize=12)
plt.ylabel('Total Sales ($)', fontsize=12)
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

for container in ax.containers:
    ax.bar_label(container, fmt='$%.0fK', labels=[f'${v/1000:.0f}K' for v in container.datavalues], fontsize=8)

plt.tight_layout()
plt.show()

Looks like there's a trend of rising sales in Q3 over Q1, Q2 sales. Then sales are doubled in Q4 over Q3 sales

Also there's a rising in sales from 2003 to 2004 in all quarters

In [ ]:
pivot_data = sales_per_quarter.pivot(index='QTR_ID', columns='YEAR_ID', values='SALES')
pivot_data['Growth_2003_to_2004_%'] = ((pivot_data[2004] - pivot_data[2003]) / pivot_data[2003]) * 100
pivot_data['Growth_2004_to_2005_%'] = ((pivot_data[2005] - pivot_data[2004]) / pivot_data[2004]) * 100
pivot_data

growth shown from year to year 2003-2004 in all quarters, and especially to be huge rise in Q1 and Q3.

In [ ]:
sales_per_month = sales_table.groupby(['YEAR_ID', 'MONTH_ID'])['SALES'].sum().reset_index()
sales_per_month

In [ ]:
sales_per_month['Month'] = 'M' + sales_per_month['MONTH_ID'].astype(str)

plt.figure(figsize=(12, 6))
ax = sns.barplot(data=sales_per_month, x='Month', y='SALES', hue='YEAR_ID', palette='Set2')

plt.title('Monthly Sales: Year-over-Year Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Sales ($)', fontsize=12)
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

for container in ax.containers:
    ax.bar_label(container, fmt='$%.0fK', labels=[f'${v/1000:.0f}K' for v in container.datavalues], fontsize=8)

plt.tight_layout()
plt.show()

Shown that 2055 is rising above 2003, 2004 in all months

Looks like October and November are the best selling months. With November scoring double sales than October.

In [ ]:
sales_per_country = sales_table.groupby(['COUNTRY'])['SALES'].sum().sort_values(ascending=False)
sales_per_country

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=sales_per_country)
plt.title('')
plt.xticks(rotation=90)

for i, (country, sales) in enumerate(sales_per_country.items()):
    plt.text(i, sales, f"${sales/1000:,.0f}k",
    ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

Looks like the biggest market is USA with 3.628m

Then outside USA, come Spain (1.216m) and France (1.111m)

In [ ]:
sales_per_city = sales_table.groupby(['CITY'])['SALES'].sum().sort_values(ascending=False).head(20)
sales_per_city

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=sales_per_city)
plt.title('')
plt.xticks(rotation=90)

for i, (city, sales) in enumerate(sales_per_city.items()):
    plt.text(i, sales, f"${sales/1000:,.0f}k",
    ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

Madrid comes with the highest source of revenue with 1.083m

Then comes San Rafael (655k) and NYC (561k)

In [ ]:
total_sales = sales_table['SALES'].sum()
sales_per_customer = sales_table.groupby(['CUSTOMERNAME'])['SALES'].sum().sort_values(ascending=False).head(20)
sales_per_customer

In [ ]:
sales_per_customer_share = (sales_per_customer/total_sales)*100
sales_per_customer_share

Looks like around 9% of sales are coming from Euro Shopping Channel, and around 6.5% coming from Mini Gifts Distributors Ltd.

Seems not so high dependecy on a particular customer to drive sales

In [ ]:
sales_per_product_count = sales_table['PRODUCTLINE'].value_counts().sort_values(ascending=False)
sales_per_product_count

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=sales_per_product_count)
plt.title('')
plt.xticks(rotation=90)

for i, (product, sales) in enumerate(sales_per_product_count.items()):
    plt.text(i, sales, f"${sales:,.0f}",
    ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
sales_per_product_sales = sales_table.groupby(['PRODUCTLINE'])['SALES'].sum().sort_values(ascending=False)
sales_per_product_sales

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=sales_per_product_sales)
plt.title('')
plt.xticks(rotation=90)

for i, (product, sales) in enumerate(sales_per_product_sales.items()):
    plt.text(i, sales, f"${sales/1000:,.0f}k",
    ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

Looks like Classic cars are the main sales source with 3.920m

In [ ]:
sales_per_product_share = (sales_per_product/total_sales)*100
sales_per_product_share